In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://bestpoolshop.com/product-category/pool-chemicals/chlorine-bromine-tablets/"
headers = {"User-Agent": "Mozilla/5.0"}

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

product_links = []

# Select direct anchor tags inside each product item only once
product_items = soup.select("ul.products li.product")

for item in product_items:
    a_tag = item.find("a", class_="woocommerce-LoopProduct-link")
    if a_tag and a_tag.get("href"):
        product_links.append(a_tag["href"])

print(f"🔗 Total Unique Product Links Found: {len(product_links)}")
for link in product_links:
    print(link)


🔗 Total Unique Product Links Found: 16
https://bestpoolshop.com/product/poolife-nst-non-stabilized-swimming-pool-chlorine-tablet-20-6lb-22421/
https://bestpoolshop.com/product/poolife-nst-non-stabilized-swimming-pool-chlorine-tablet-44lb-22422/
https://bestpoolshop.com/product/50lb-3inch-chlorine-tablets-for-swimming-pools-10-buckets-2/
https://bestpoolshop.com/product/clearview-bromo-bromine-brominating-1in-tablets-4lb-cvbr004/
https://bestpoolshop.com/product/cvtlst005-clearview-swimming-pool-spa-chlorine-chlorinating-3-inch-tablets-pucks-tabs-5-lbs/
https://bestpoolshop.com/product/cvtlst010-clearview-swimming-pool-spa-chlorine-chlorinating-3-inch-tablets-pucks-tabs-10-lbs/
https://bestpoolshop.com/product/clearview-swimming-pool-spa-chlorine-chlorinating-1-inch-tablets-pucks-tabs-5-lbs-cvts005/
https://bestpoolshop.com/product/swimming-pool-3-inch-chlorine-tablets-25lb-bucket/
https://bestpoolshop.com/product/50lb-3inch-chlorine-tablets-for-swimming-pools/
https://bestpoolshop.com/

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Headers for better site compatibility
headers = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/122.0.0.0 Safari/537.36"),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive"
}

base_url = "https://bestpoolshop.com"
list_url = f"{base_url}/product-category/pool-chemicals/chlorine-bromine-tablets/"

def fetch_with_retries(url, retries=3, delay=1):
    """Attempt to fetch a URL with retries."""
    for attempt in range(1, retries + 1):
        try:
            resp = requests.get(url, headers=headers, timeout=10)
            resp.raise_for_status()
            return resp
        except Exception as e:
            print(f"⚠️ Attempt {attempt}/{retries} failed: {url}")
            if attempt < retries:
                time.sleep(delay)
    return None

# Step 1: Get unique product links
resp = fetch_with_retries(list_url)
soup = BeautifulSoup(resp.text, "html.parser") if resp else None

product_links = []
if soup:
    anchors = soup.select("ul.products li.product a.woocommerce-LoopProduct-link")
    product_links = [a["href"] for a in anchors if a.get("href")]
    product_links = list(dict.fromkeys(product_links))

print(f"✅ Total unique product links: {len(product_links)}")

# Step 2: Scrape each product page
all_data = []
failed_urls = []

for link in product_links:
    full_url = link if link.startswith("http") else base_url + link
    res = fetch_with_retries(full_url)
    if not res:
        failed_urls.append(full_url)
        continue

    soup = BeautifulSoup(res.text, "html.parser")

    title = soup.select_one("h1.product_title")
    title = title.get_text(strip=True) if title else ""

    overview_ul = soup.select_one("#cgkit-tab-description ul")
    overview = ", ".join([li.get_text(strip=True) for li in overview_ul.select("li")]) if overview_ul else ""

    price_tag = soup.select_one("p.price ins bdi") or soup.select_one("p.price bdi")
    price = price_tag.get_text(strip=True) if price_tag else ""

    in_store_tag = soup.select_one("div.in-store-price span")
    in_store_price = in_store_tag.get_text(strip=True).replace('in Store', '').strip() if in_store_tag else ""


    stock_tag = soup.select_one("p.stock")
    stock = stock_tag.get_text(strip=True) if stock_tag else "N/A"

    desc_heading = soup.select_one("#cgkit-tab-description > h2")
    description = desc_heading.get_text(strip=True) if desc_heading else ""

    pid_tag = soup.select_one("p.product_id small")
    product_id = pid_tag.get_text(strip=True).replace("Product ID:", "").strip() if pid_tag else ""

    data_dict = {
        "title": title,
        "overview": overview,
        "price": price,
        "in_store_price": in_store_price,
        "stock": stock,
        "description": description,
        "link": full_url,
        "product_id": product_id
    }

    # Extract <li> key-value pairs only from the description section
    desc_block = soup.select_one("div#cgkit-tab-description")
    if desc_block:
        for li in desc_block.select("ul li"):
            text = li.get_text(strip=True)
            if ":" in text:
                key, value = map(str.strip, text.split(":", 1))
                data_dict[key] = value.rstrip(',')  # Remove trailing commas if present

    all_data.append(data_dict)



    print(f"✅ Scraped: {title}")
    time.sleep(1)

# Step 3: Convert to DataFrame
df = pd.DataFrame(all_data)
df.head(), df.shape

# Report failures
if failed_urls:
    print(f"\n❌ Failed to fetch {len(failed_urls)} pages:")
    for url in failed_urls:
        print("   ↪", url)


✅ Total unique product links: 16
✅ Scraped: 22421 Poolife NST Non Stabilized Swimming Pool Chlorine Tablet 20.6lb
✅ Scraped: 22422 Poolife NST Non Stabilized Swimming Pool Chlorine Tablet 44lb
✅ Scraped: Bulk 50lb 3inch Chlorine Tablets For Swimming Pools – 10 Buckets
✅ Scraped: CVBR004 Clearview Bromo Bromine Brominating 1in. Tablets 4lb.
✅ Scraped: CVTLST005 ClearView Swimming Pool Spa Chlorine Chlorinating 3 inch Tablets Pucks Tabs 5 lbs.
✅ Scraped: CVTLST010 ClearView Swimming Pool Spa Chlorine Chlorinating 3 inch Tablets Pucks Tabs 10 lbs.
✅ Scraped: CVTS005 ClearView Swimming Pool Spa Chlorine Chlorinating 1 inch Tablets Pucks Tabs 5 lbs.
✅ Scraped: Pool Chlorine Tablets 25lb 3 inch Jumbo
✅ Scraped: Pool Chlorine Tablets 50lb for 3 inch Jumbo ClearView
✅ Scraped: 42104 Swimming Pool Chlorine Chlorinating 1 inch Tablets Pucks Tabs 5 lbs.
✅ Scraped: 62122 Hasa Small Chlorinating Tablets 2 lbs.
✅ Scraped: 62122 Hasa Small Chlorinating Tablets 2 lbs. 2-Pack Multi-Pack
✅ Scraped: 6308

In [2]:
df

,title,overview,price,in_store_price,stock,description,link,product_id,Calcium Hypochlorite,Other Ingredients,Trichlor-s-Triazinetrione,Available Chlorine,ACTIVE INGREDIENT
0,22421 Poolife NST Non Stabilized Swimming Pool...,Use only in pools with a skimmer and skimmer b...,$185.76,$171.99,Only 7 left in stock,Poolife NST Non Stabilized Without Conditioner...,https://bestpoolshop.com/product/poolife-nst-n...,36140,70.2%,29.8%,NaN,NaN,NaN
1,22422 Poolife NST Non Stabilized Swimming Pool...,Use only in pools with a skimmer and skimmer b...,,$234.99,1 in stock,Poolife NST Non Stabilized Without Conditioner...,https://bestpoolshop.com/product/poolife-nst-n...,80944,70.2%,29.8%,NaN,NaN,NaN
2,Bulk 50lb 3inch Chlorine Tablets For Swimming ...,Do not allow this product to get damp or wet b...,,$1500.00,2 in stock,Swimming Pool 3 inch Stabilized Chlorine Table...,https://bestpoolshop.com/product/50lb-3inch-ch...,110412,NaN,1%,99%,90%,NaN
3,CVBR004 Clearview Bromo Bromine Brominating 1i...,"96% Bromine, Slow dissolving, Reduced odor, En...",$71.99,$64.99,Only 8 left in stock,Oreq Clearview Swimming Pool and Spa Bromo Bro...,https://bestpoolshop.com/product/clearview-bro...,119703,NaN,NaN,NaN,NaN,"1-Bromo-3-chloro-5,5-dimethylhydantoin – 96.0%"
4,CVTLST005 ClearView Swimming Pool Spa Chlorine...,World’s First Low-Odor Chlorine TabsExperience...,$48.99,$41.99,Only 3 left in stock,CVTLST005 Scent-Trific ClearView Swimming Pool...,https://bestpoolshop.com/product/cvtlst005-cle...,137117,NaN,NaN,NaN,NaN,NaN
5,CVTLST010 ClearView Swimming Pool Spa Chlorine...,World’s First Low-Odor Chlorine TabsExperience...,$86.99,$67.99,Only 2 left in stock,CVTLST010 Scent-Trific ClearView Swimming Pool...,https://bestpoolshop.com/product/cvtlst010-cle...,137128,NaN,NaN,NaN,NaN,NaN
6,CVTS005 ClearView Swimming Pool Spa Chlorine C...,,$49.99,$42.99,In stock,Oreq ClearView Swimming Pool Spa Chlorine Chlo...,https://bestpoolshop.com/product/clearview-swi...,118986,NaN,NaN,NaN,NaN,NaN
7,Pool Chlorine Tablets 25lb 3 inch Jumbo,,,$114.99,11 in stock,Swimming Pool 3 inch Chlorine Tablets 25lb Bucket,https://bestpoolshop.com/product/swimming-pool...,65656,NaN,NaN,NaN,NaN,NaN
8,Pool Chlorine Tablets 50lb for 3 inch Jumbo Cl...,Do not allow this product to get damp or wet b...,,$179.99,8 in stock,Swimming Pool 3 inch Stabilized Chlorine Table...,https://bestpoolshop.com/product/50lb-3inch-ch...,80282,NaN,1%,99%,90%,NaN
9,42104 Swimming Pool Chlorine Chlorinating 1 in...,"Four one-half ounce tablets per 10,000 gallons...",$64.99,,Out of stock,Poolife Small 1 inch Swimming Pool and Spa Chl...,https://bestpoolshop.com/product/poolife-small...,92700,NaN,NaN,NaN,NaN,NaN


In [3]:
all_data

[{'title': '22421 Poolife NST Non Stabilized Swimming Pool Chlorine Tablet 20.6lb',
  'overview': 'Use only in pools with a skimmer and skimmer basket., Skimmer basket should be free of all other water treatment., Place tablet in the empty skimmer basket., Replace with a new tablet as needed. Do not mix with other products or dissolve before use.',
  'price': '$185.76',
  'in_store_price': '$171.99',
  'stock': 'Only 7 left in stock',
  'description': 'Poolife NST Non Stabilized Without Conditioner Swimming Pool Chlorine Tablet 20.6lb 22421',
  'link': 'https://bestpoolshop.com/product/poolife-nst-non-stabilized-swimming-pool-chlorine-tablet-20-6lb-22421/',
  'product_id': '36140',
  'Calcium Hypochlorite': '70.2%',
  'Other Ingredients': '29.8%'},
 {'title': '22422 Poolife NST Non Stabilized Swimming Pool Chlorine Tablet 44lb',
  'overview': 'Use only in pools with a skimmer and skimmer basket., Skimmer basket should be free of all other water treatment., Place tablet in the empty ski

In [5]:
df['Manufacturer SKU'] = df['title'].apply(lambda x: x.split()[0] if isinstance(x, str) and x.split() and any(c.isdigit() for c in x.split()[0]) else "")

In [6]:
df.head()

,title,overview,price,in_store_price,stock,description,link,product_id,Calcium Hypochlorite,Other Ingredients,Trichlor-s-Triazinetrione,Available Chlorine,ACTIVE INGREDIENT,Manufacturer SKU
0,22421 Poolife NST Non Stabilized Swimming Pool...,Use only in pools with a skimmer and skimmer b...,$185.76,$171.99,Only 7 left in stock,Poolife NST Non Stabilized Without Conditioner...,https://bestpoolshop.com/product/poolife-nst-n...,36140,70.2%,29.8%,NaN,NaN,NaN,22421
1,22422 Poolife NST Non Stabilized Swimming Pool...,Use only in pools with a skimmer and skimmer b...,,$234.99,1 in stock,Poolife NST Non Stabilized Without Conditioner...,https://bestpoolshop.com/product/poolife-nst-n...,80944,70.2%,29.8%,NaN,NaN,NaN,22422
2,Bulk 50lb 3inch Chlorine Tablets For Swimming ...,Do not allow this product to get damp or wet b...,,$1500.00,2 in stock,Swimming Pool 3 inch Stabilized Chlorine Table...,https://bestpoolshop.com/product/50lb-3inch-ch...,110412,NaN,1%,99%,90%,NaN,
3,CVBR004 Clearview Bromo Bromine Brominating 1i...,"96% Bromine, Slow dissolving, Reduced odor, En...",$71.99,$64.99,Only 8 left in stock,Oreq Clearview Swimming Pool and Spa Bromo Bro...,https://bestpoolshop.com/product/clearview-bro...,119703,NaN,NaN,NaN,NaN,"1-Bromo-3-chloro-5,5-dimethylhydantoin – 96.0%",CVBR004
4,CVTLST005 ClearView Swimming Pool Spa Chlorine...,World’s First Low-Odor Chlorine TabsExperience...,$48.99,$41.99,Only 3 left in stock,CVTLST005 Scent-Trific ClearView Swimming Pool...,https://bestpoolshop.com/product/cvtlst005-cle...,137117,NaN,NaN,NaN,NaN,NaN,CVTLST005


In [7]:
df.to_csv('bestpoolshop_chlorine.csv', index=False)